In [4]:
import pandas as pd

In [75]:
df=pd.read_csv("data/datasets_overview.csv")

In [90]:
import spacy
from rank_bm25 import BM25Okapi
from tqdm import tqdm
nlp = spacy.load("de_core_news_lg")
text_list = df["dataset_title"].str.lower().values
text_list=np.unique(text_list)
tok_text=[] # for our tokenised corpus
#Tokenising using SpaCy:
for doc in tqdm(nlp.pipe(text_list, disable=["tagger", "parser","ner"])):
    tok = [t.text for t in doc if t.is_alpha]
    tok_text.append(tok)

340it [00:00, 2833.30it/s]


In [91]:
text_list

array(['1-personenhaushalte [anz.]', '15-jahresverbrauch  [%]',
       '15-jahresverbrauch [ha]', '2-personenhaushalte [anz.]',
       '3-personenhaushalte [anz.]', '4-personenhaushalte [anz.]',
       '5-personenhaushalte [anz.]',
       '6 und mehr personenhaushalte [anz.]',
       'abstimmungsarchiv des kantons zürich', 'aerzte [anz.]',
       'aerzte [pro 1000 einw.]', 'altersquotient [%]',
       'altpapiermengen [tonnen]',
       'anfragestatistik des data shop im statistischen amt des kantons zürich',
       'anteil 1 zi.-wohnungen [%]', 'anteil 1-personenhaushalte [%]',
       'anteil 2 zi.-wohnungen [%]', 'anteil 2-personenhaushalte [%]',
       'anteil 3 zi.-wohnungen [%]', 'anteil 3-personenhaushalte [%]',
       'anteil 4 zi.-wohnungen [%]', 'anteil 4-personenhaushalte [%]',
       'anteil 5 zi.-wohnungen [%]', 'anteil 5-personenhaushalte [%]',
       'anteil 6 und mehr personenhaushalte [%]',
       'anteil 6+ zi.-wohnungen [%]', 'anteil efh am wohnungsbestand [%]',
      

In [92]:
bm25 = BM25Okapi(tok_text)

In [96]:
query = "Eigenkapital"
tokenized_query = query.lower().split(" ")
import time
t0 = time.time()
results = bm25.get_top_n(tokenized_query, text_list, n=5)
t1 = time.time()
print(f'Searched in {round(t1-t0,3) } seconds \n')

for i in results:
    print(i)

Searched in 0.001 seconds 

eigenkapital [fr./einw.]
eigenkapital steuerpfl. jur. pers. [mio.fr.]
ø eigenkapital steuerpfl. jur. pers. [1000 fr.]
übersicht über alle lernenden im kanton zürich
covid_19 contact tracing kanton zürich
